In [2]:
import pandas as pd

data = pd.read_csv('siamese_data.csv')
# data.head(2)
new_data = pd.DataFrame()
new_data['sentence_1'] = data['title_1'].str.lower()
new_data['sentence_2'] = data['title_2'].str.lower()
new_data['similarity'] = data['label']
new_data.head()

new_data.to_csv('custom_data_lower_classification.csv',index=False)

train_data = pd.read_csv('custom_data_lower_classification.csv')
train_data.head(2)

# data = pd.read_csv('data/train.csv')
# new_data = pd.DataFrame()
# new_data['posting_id'] = data['posting_id']
# new_data['sentence'] = data['title'].str.lower()
# new_data.to_csv('custom_train_data.csv',index=False)

,sentence_1,sentence_2,similarity
0,paper bag victoria secret,paper bag victoria secret,1
1,paper bag victoria secret,bgsr jaket cagoule / jaket cagoule premium / j...,0


In [3]:
# import tarfile
# train_data.shape
x = 0
for i in range(train_data.shape[0]):
    for j in range(train_data.shape[0]):
        x += 1
x

3103158436

In [9]:
# temp = tarfile.open('web_bert_similarity.tar.gz')
import numpy as np
# np.ones((int(train_data.shape[0]/2),int(train_data.shape[0]/2)))

In [4]:
# temp.extractall()

In [5]:
# from pytorch_transformers import BertTokenizer

In [6]:
# bert_tokenizer = BertTokenizer.from_pretrained('./')

In [7]:
import torch


from learner import TextSimilarityLearner
from dataset import train_test_val

learner = TextSimilarityLearner(batch_size=1,
                                model_name='web-bert-similarity',
                                loss_func=torch.nn.MSELoss(),
                                learning_rate=5e-5,
                                weight_decay=0,
                                device=torch.device('cuda:0'))

train_dataset, eval_dataset,_ = train_test_val(learner.bert_tokenizer, path='D:\GitHub\Image_matching\custom_data.csv')

learner.load_train_data(train_dataset)
learner.train(epoch=1)

  0%|                                                                                        | 0/31194 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 1.21 GiB already allocated; 8.82 MiB free; 1.34 GiB reserved in total by PyTorch)

In [ ]:
!pip install GPUtil

from GPUtil import showUtilization as gpu_usage
gpu_usage()                             

In [ ]:
# import torch
# from GPUtil import showUtilization as gpu_usage
# from numba import cuda

# def free_gpu_cache():
#     print("Initial GPU Usage")
#     gpu_usage()                             

#     torch.cuda.empty_cache()

#     cuda.select_device(0)
#     cuda.close()
#     cuda.select_device(0)

#     print("GPU Usage after emptying the cache")
#     gpu_usage()

# free_gpu_cache() 

In [ ]:
from torch.utils.data import DataLoader

def Train(train_data,batch_size = 10,
          model_name = 'web-bert-similarity',
          loss_fn = torch.nn.MSELoss(),
          learning_rate = 1e-5,
          weight_decay = 0,
          epochs = 10
          device = torch.device('cuda:0')):
    
    train_data = DataLoader(train_data,batch_size=batch_size,shuffle=True)
    
    optimizer = torch.optim.Adam(model.parameters(), weight_decay=weight_decay, lr=learning_rate)
    
    bert_tokenizer, model = bert_similarity('')
    
    total = len(train_data)
    
    model.to(device)
    model.train()
    for i in range(epochs):
        epoch_loss = .0
        
        pbar = tqdm(total=total)
        
        for i, (dataset_input_ids, dataset_token_type_ids, dataset_attention_masks,
                dataset_scores) in enumerate(train_data):
            dataset_input_ids = dataset_input_ids.to(device)
            dataset_token_type_ids = dataset_token_type_ids.to(device)
            dataset_attention_masks = dataset_attention_masks.to(device)
            dataset_scores = dataset_scores.to(device)
            
            optimizer.zero_grad()
        
            outputs = model(dataset_input_ids, dataset_token_type_ids, dataset_attention_masks).squeeze(-1)
            step_loss = loss_func(outputs, dataset_scores)
            step_loss.backward()
            optimizer.step()
            epoch_loss += step_loss.item()
            
            pbar.update(1)
            pbar.set_postfix(step_loss=step_loss)
            
        pbar.close()
        
        matric =  {'loss': epoch_loss / total}
        log_info = 'Epoch {}: '.format(i + 1)
        for key, value in matric.items():
                log_info += key + '=' + str(value) + ' '
        print(log_info)
    